# Cookie / Session

## Cookie 이용 방식

### Before

In [1]:
import requests

url = 'http://pythonscraping.com/pages/cookies/welcome.php'

html = requests.post(url, {'username':'test','password':'password'})
html.text 

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

### After

In [3]:
html.cookies.get_dict()

{'loggedin': '1', 'username': 'test'}

In [4]:
html = requests.get(url, cookies=html.cookies)
html.text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

## Session 이용 방식

In [5]:
session = requests.Session()

In [6]:
html = session.post(url, {'username':'test','password':'password'})

In [7]:
html = session.post(url)
html.text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

## 정규식

Meta characters

- . => Any character(single)
- [or] => brakey(범위) one of them
- ? => True or False
- {m,n} => min=m, max=n
- '*' => {0,}
- '+' => {1,}
- \d => 숫자
- \w => A-Za-z0-9_
- \s => whitespace
- \b => Word Boundary
- \D => !숫자
- \W => 문자가 아닌 애들
- \S => 공백이 아닌 애들
- \B => \b가 아닌 애들(글자 사이에 있는 애들)
- .+ => 아무글자가 1번(Greedy)
- .+? => Lazy
- r'' => raw format
- () => group

In [8]:
import re

re.findall('.','가나다라마바사'), \
re.findall('가.다','가나다라마바사 가가다 가다 가라다 가가가가다'), \
re.findall('아버지.', '아버지가 아버지께서 아버지와 아버지는'), \
re.findall('아.?지', '아버지가 아버지께서 아버지와 아버지는 강아지'), \
re.findall('아.+?지', '아버지가 아버지께서 아버지와 아버지는 강아지'), \
re.findall('아{2,5}지', '아버지가 아버지께서 아버지와 아버지는 강아지 아아아아지'), \
re.findall('[A-Z]', '아버지가 아버지께서 아버지와 아버지는 강AS아SDF지'), \
re.findall('[ㄱ-ㅎ]', 'ㄱㄴㅎㄷㅇ 아버지가 아버지께서 아버지와 아버지는 강AS아SDF지'), \
re.findall('[ㅏ-ㅣ]{2,}', 'ㅠㅠ ㅡㅡ ㅣㅣ 아버지'), \
re.findall('[ㄱ-ㅎ]{2,}', 'ㅅㅂ ㅋㅋ ㅎㅎ'), \
re.findall(r'\b[가-힣]+\b', '아버^^지가 아버지께서 아버지와 아버지는')

(['가', '나', '다', '라', '마', '바', '사'],
 ['가나다', '가가다', '가라다', '가가다'],
 ['아버지가', '아버지께', '아버지와', '아버지는'],
 ['아버지', '아버지', '아버지', '아버지', '아지'],
 ['아버지', '아버지', '아버지', '아버지'],
 ['아아아아지'],
 ['A', 'S', 'S', 'D', 'F'],
 ['ㄱ', 'ㄴ', 'ㅎ', 'ㄷ', 'ㅇ'],
 ['ㅠㅠ', 'ㅡㅡ', 'ㅣㅣ'],
 ['ㅅㅂ', 'ㅋㅋ', 'ㅎㅎ'],
 ['아버', '지가', '아버지께서', '아버지와', '아버지는'])

In [9]:
import requests
from bs4 import BeautifulSoup
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}

In [10]:
def getDownload(url, param = None, retries = 3):
    resp = None
    try:
        resp = requests.get(url, params = param, headers = header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries : {0}'.format(retries))
            return getDownload(url, param, retries -1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            
    return resp

In [11]:
url = 'http://example.webscraping.com/places/default/index'
resp = getDownload(url)

In [12]:
re.findall(r'[a-z][a-z0-9]{2,}@','test@domain.com'), \
re.findall(r'([a-z][a-z0-9]{2,})@(\w{3,}[.][a-z]{3}$)','test@domain.com'), \
re.findall(r'([a-z][a-z0-9]{2,})@(\w{3,}([.][a-z]{2,})+)','test@domain.co.kr')

(['test@'], [('test', 'domain.com')], [('test', 'domain.co.kr', '.kr')])

In [13]:
url = 'https://www.google.com/search'
params = {
    'q':'성소'
}
resp = getDownload(url, params)

In [14]:
result = re.search(r'<title>(.+)</title>', resp.text)
result.groups(1)

('성소 - Google 검색',)

In [15]:
from html import unescape
result = re.findall(r'<a href="(.+?)".+?><h3 class="LC20lb"><div class="ellip">(.+?)</div>', resp.text)
[(requests.compat.urljoin(resp.url, _[0]), unescape(_[1])) for _ in result]

[('https://www.google.com/search?q=%EC%84%B1%EC%86%8C&amp;gbv=1&amp;sei=GfuKXcDpHIj_wAPQ0YawCA',
  '성소(우주소녀) - 나무위키'),
 ('https://namu.wiki/w/%EC%84%B1%EC%86%8C', '성소 - 나무위키'),
 ('https://ko.wikipedia.org/wiki/%EC%84%B1%EC%86%8C_(%EA%B0%80%EC%88%98)',
  '성소 (가수) - 위키백과, 우리 모두의 백과사전'),
 ('https://1boon.daum.net/benter/sungso', '비주얼 파티 중이라는 우주소녀 성소 근황 | 1boon'),
 ('http://news.chosun.com/site/data/html_dir/2018/09/03/2018090301347.html',
  '[SC이슈]"컴백 NO, 예능은 OK"…우주소녀 성소, 中스케줄 논란 ...'),
 ('https://news.sbs.co.kr/news/endPage.do?news_id=N1004917577',
  '우주소녀, 13명→10명 체제 컴백…"성소 등 3명은 참여NO"'),
 ('https://1boon.kakao.com/newsade/WJSN-SS', '우주소녀 성소, CG 아닌가요? | 1boon'),
 ('https://www.msn.com/ko-kr/entertainment/music/%EC%9A%B0%EC%A3%BC%EC%86%8C%EB%85%80-wjsn-%EC%84%B1%EC%86%8C-%EC%A4%91%EA%B5%AD%EC%97%90%EC%84%9C%EC%9D%98-%EA%B7%BC%ED%99%A9-%EB%88%88%EA%B8%B8-%E2%80%98%EB%8D%94-%EC%98%88%EB%BB%90%EC%A7%84-%EC%99%B8%EB%AA%A8%E2%80%99/ar-BBRn6hi',
  "우주소녀(WJSN) 성소, 중국에서의 근황 눈길 '더 예뻐진 외모'")]